In [34]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [35]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

896438
887257


In [36]:
df = df.with_columns(pl.col('100_a').map_elements(hezke_jmeno, return_dtype=str).alias('jmeno'))

In [37]:
bio = pl.read_parquet(os.path.join('data','narozeni-umrti-gender.parquet'))

In [38]:
df = df.join(bio, on='100_7', how='left')

In [39]:
df.filter(pl.col('stran') >= 50).with_columns(pl.col(pl.String).str.replace("Kytice z pověstí národních","Kytice")).group_by(
    ['jmeno','245_a','245_p']
).len().sort(by='len',descending=True).head(30).join(
    df.with_columns(pl.col(pl.String).str.replace("Kytice z pověstí národních","Kytice")).group_by(['jmeno','245_a']).agg(pl.col('rok').min()
), on=['245_a','jmeno'], how='left')

jmeno,245_a,245_p,len,rok
str,str,str,u32,i64
null,"""Divadelní ochotník""",null,216,null
"""Božena Němcová""","""Babička""",null,216,1855
"""Karel Jaromír Erben""","""Kytice""",null,144,1853
"""Karel Hynek Mácha""","""Máj""",null,142,1872
null,"""Čítanka pro školy obecné""",null,110,null
null,"""Russkij jazyk""",null,96,null
"""Jan Karafiát""","""Broučci""",null,91,1876
"""Alois Jirásek""","""Staré pověsti české""",null,89,1894
"""Jan Neruda""","""Povídky malostranské""",null,78,1878


In [89]:
topka = [
    ["Božena Němcová","Babi"],
    ["Karel Hynek Mácha","Má[jg]"],
    ["Karel Jaromír Erben","(?i)Kyti"],
    ["Alois Jirásek","(?i)pověst"],
    ["Jan Neruda","(?i)malostrans"],
    ["Alois Jirásek","Psohlavci"],
    ["Božena Němcová","(?i)pohorsk"],
    ["Antoine de Saint-Exupéry","Malý princ"],
    ["Karel Tůma","Z českých mlýnů"],
    ["Karel Čapek","(?i)válka s mloky"],
    ["Alexandre Dumas","Tři mušketýři"],
    ["Karel Poláček","Bylo nás pět"],
    ["Petr Bezruč","Slezské písně"],
    ["Jaroslav Hašek", "Švejk"],
    ["Jan Karafiát","(?i)brouč"],
    ["Karel Václav Rais","(?i)vlastenc"	]
]

In [84]:
len(topka)

16

In [91]:
top_vyber = pl.DataFrame()
for autor_titul in topka:
    vyber = df.filter(~pl.col("100_7").is_null()).filter((pl.col("jmeno").str.contains(autor_titul[0])) & (pl.col('245_a').str.contains(autor_titul[1])))
    pravy_titul = vyber.sort(by='rok').select(pl.col('245_a')).to_series().to_list()[0].split(' - ')[0].replace('a jiné podivné historky','').replace("Mág","Máj").strip()
    pocet = len(vyber)
    hezky_nazev = f"{pocet}× {autor_titul[0].split(' ')[-1]}: {pravy_titul}"
    top_vyber = pl.concat(
        [top_vyber, vyber.with_columns(
            pl.lit(hezky_nazev).alias('kniha')).with_columns(
            pl.lit(pocet).alias('kolik')
        ).select(pl.col(['rok','kniha','kolik','umrti','jmeno']))
        ])

top_vyber = top_vyber.with_columns(pl.col('umrti').map_elements(
            lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date
        )
        .cast(pl.Datetime))
poradi_vyber = top_vyber.group_by('kniha').len().sort(by='len',descending=True).select(pl.col('kniha')).to_series().to_list()
poradi_pocet = top_vyber.group_by('kniha').len().sort(by='len',descending=True).select(pl.col('len')).to_series().to_list()
print(poradi_vyber)

['235× Němcová: Babička', '221× Mácha: Máj', '188× Erben: Kytice z pověstí národních', '127× Karafiát: Broučci', '114× Neruda: Povídky malostranské', '111× Hašek: Dobrý voják Švejk', '90× Jirásek: Staré pověsti české', '88× Dumas: Tři mušketýři', '71× Němcová: Pohorská vesnice', '66× Jirásek: Psohlavci', '50× Saint-Exupéry: Malý princ', '46× Bezruč: Slezské písně', '45× Tůma: Z českých mlýnů', '43× Čapek: Válka s Mloky', '42× Rais: Zapadlí vlastenci', '41× Poláček: Bylo nás pět']


In [76]:
top_vyber.sample(10)

rok,kniha,kolik,umrti,jmeno
i64,str,i32,datetime[μs],str
1911,"""188× Erben: Kytice z pověstí n…",188,1870-01-01 00:00:00,"""Karel Jaromír Erben"""
1916,"""221× Mácha: Máj""",221,1836-01-01 00:00:00,"""Karel Hynek Mácha"""
1942,"""45× Tůma: Z českých mlýnů""",45,1917-01-01 00:00:00,"""Karel Tůma"""
1944,"""71× Němcová: Pohorská vesnice""",71,1862-01-01 00:00:00,"""Božena Němcová"""
1894,"""71× Němcová: Pohorská vesnice""",71,1862-01-01 00:00:00,"""Božena Němcová"""
1914,"""114× Neruda: Povídky malostran…",114,1891-01-01 00:00:00,"""Jan Neruda"""
1937,"""235× Němcová: Babička""",235,1862-01-01 00:00:00,"""Božena Němcová"""
2017,"""88× Dumas: Tři mušketýři""",88,1870-01-01 00:00:00,"""Alexandre Dumas"""
1942,"""45× Tůma: Z českých mlýnů""",45,1917-01-01 00:00:00,"""Karel Tůma"""


In [93]:
base = alt.Chart(
    alt_friendly(top_vyber),
    title=alt.Title(
        ['Kdy vycházely nejvydávanější autorské knihy'],
        subtitle = ['Růžové tečky jsou knihy české, modré překladové. Čárka označuje rok úmrtí.']
    )
)

kdy_umreli = alt.Chart(alt_friendly(top_vyber)).mark_tick(
    color='#292829',  # optional: you can specify color
    thickness=1.75,
    height=10,
    opacity=0.2
).encode(
    x=alt.X('umrti:T', title=None),
    y=alt.Y("kniha:N", sort=poradi_vyber, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white', labelExpr='split(datum.label, "× ")[1]')))

domaci = base.mark_circle(size=9, opacity=1).encode(
    x=alt.X('rok:T', title=None), 
    y=alt.Y('kniha:N', sort=poradi_vyber, title=None, axis=alt.Axis(orient='left', tickColor='white', labels=False)),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    color=alt.Color('kniha:N', scale=alt.Scale(range=['#E09DA3']), legend=None)
).transform_filter(
    (alt.datum.kniha != '88× Dumas: Tři mušketýři') & (alt.datum.kniha != '50× Saint-Exupéry: Malý princ')
).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

zahranicni = base.mark_circle(size=9, color='#81A9D5').encode(
    x=alt.X('rok:T', title=None), 
    y=alt.Y('kniha:N', sort=poradi_vyber, title=None, axis=alt.Axis(orient='left',tickColor='white', labels=False)),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    color=alt.Color('kniha:N', scale=alt.Scale(range=['#81A9D5']), legend=None)
).transform_filter(
    (alt.datum.kniha == '88× Dumas: Tři mušketýři') | (alt.datum.kniha == '50× Saint-Exupéry: Malý princ')
).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

pocty = base.mark_circle(size=0).encode(x=alt.X('rok:T', title=None), 
        y=alt.Y('kniha:N', sort=poradi_pocet, title=None, axis=alt.Axis(orient="right", tickColor='white', labelExpr='split(datum.label, " ")[0]')))

nejvydavanejsi = alt.layer(kdy_umreli, domaci, zahranicni, pocty).properties(
    width=kredity['sirka'] * 1.15, autosize={'type': 'fit', 'contains': 'padding'}
).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared").resolve_axis(y='independent')

nejvydavanejsi

alt.LayerChart(...)

In [95]:
me_to_neurazi(nejvydavanejsi, soubor="01_nejvydavanejsi_knihy", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/01_nejvydavanejsi_knihy.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/01_nejvydavanejsi_knihy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


## Seriály

In [96]:
df = df.with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))

In [99]:
df.group_by("245_p").len().sort(by="len",descending=True)

245_p,len
str,u32
null,836219
"""Místopis Moravy""",107
"""Vlastní státní správa""",87
"""Matematika""",71
"""Text""",67
"""Geometrie""",61
"""Zvířata""",57
"""Skladba""",55
"""Správa státních podniků""",52


In [101]:
df.group_by("245_a").agg(pl.col("245_p").unique().len()).sort(by="245_p",descending=True).head(100)

245_a,245_p
str,u32
"""Sčítání lidu, domů a bytů k 1.…",307
"""Pedagogické dokumenty""",291
"""Učební dokumenty pro učební ob…",287
"""Učební dokumenty pro střední o…",271
"""Prozatimní učební dokumenty uč…",249
"""Učební plán a osnovy odborných…",248
"""Z pamětí amerického detektiva""",145
"""Přednášky Československé spole…",129
"""Star Wars""",112


## Bible

In [111]:
df.filter(pl.col("100_a").is_null() & pl.col("245_a").str.contains("(?i)bibl(ic|e|í)")).select(pl.col(['rok','245_a']))

rok,245_a
i64,str
1814,"""Biblická hystorye náboženstwj,…"
1821,"""Biblické Přjběhy starého zákon…"
1822,"""Biblické přjběhy nowého zákona…"
1823,"""Biblická hystorye náboženstwj,…"
1825,"""Náboženstwj we wýkladech a roz…"
1825,"""Náboženstwj we wýkladech a roz…"
1830,"""Biblická hystorye náboženstwj,…"
1831,"""Biblická hystorye náboženstwj,…"
1851,"""Bible česká, čili, Písmo swaté…"
